In [ ]:
# %pip install jovian

In [ ]:
# import jovian

In [ ]:
# jovian.commit(filename='data.ipynb', project='News Recommender Capstone')

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px

In [1]:
from shared_data import *

In [2]:
data = SharedData()

e:\learning\upgrad\ds\bundles\Capstone\basics\shared_data.py:16: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  self.transactions = pd.read_csv(self.txns_file_name)


In [ ]:
txns = pd.read_csv('../data/consumer_transanctions.csv')
cnt = pd.read_csv('../data/platform_content.csv')

In [ ]:
txns.head()

In [ ]:
cnt.head()

In [ ]:
# shared_data.get_data().store_transactions(txns)
# shared_data.get_data().store_content(cnt)

In [ ]:
non_eng = cnt[cnt.language != 'en']

In [ ]:
non_eng.shape

In [ ]:
txns[txns.item_id.isin(non_eng.item_id)]

### Discard articles that are not in English

In [ ]:
cnt.shape

In [ ]:
cnt = cnt[cnt.language == 'en']

In [ ]:
cnt.shape

In [ ]:
txns.shape

In [ ]:
txns = txns[~txns.item_id.isin(non_eng.item_id)]

In [ ]:
txns.shape

### Handling articles that have been pulled out

In [ ]:
cnt.interaction_type.value_counts()

In [ ]:
pulled_out = cnt[cnt.interaction_type == 'content_pulled_out']

In [ ]:
pulled_out.head()

In [ ]:
txns.columns

In [ ]:
txns.head()

In [ ]:
txns[txns.item_id.isin(pulled_out.item_id)]

### Dropping articles without title or content

In [ ]:
cnt.isna().sum()

In [ ]:
cnt.info()

In [ ]:
user_topics = []

Based on the user topics, the recommended articles change.

* At the start of the day, recommend 10 articles based on user interests
* Adjust user interests based on
    * Direct feedback
    * Clicks
    * Time spent on the article page (some threshold)
* When a new user is created, user will choose some topics
* New articles should be recommended based on collaborative and content-based filtering

### Topic modeling on articles

### Data preparation

Introduce a ratings column

In [ ]:
def to_rating(val):
    if val == 'content_followed':
        return 5
    if val == 'content_commented_on':
        return 4
    if val == 'content_saved':
        return 3
    if val == 'content_liked':
        return 2
    return 1

In [ ]:
txns.interaction_type.value_counts()

In [ ]:
txns['rating'] = txns.interaction_type.apply(lambda x: to_rating(x))

In [ ]:
txns.head()

#### Adjust IDs

In [ ]:
class IdHelper:
    _map = {}
    _id = 1

    def translate(self, id):
        # If a mapping exists for id, then return the mapping
        # Otherwise, create a new mapping, store it, and return it
        if id in self._map:
            return self._map[id]
        new_id = self.__new_id__()
        self._map[id] = new_id
        return new_id

    def __new_id__(self):
        num = self._id
        self._id += 1
        return num

In [ ]:
consumer_helper = IdHelper()
item_helper = IdHelper()

In [ ]:
txns['consumer_id_adj'] = txns['consumer_id'].map(lambda x: consumer_helper.translate(x))

In [ ]:
txns.head()

In [ ]:
txns['item_id_adj'] = txns['item_id'].map(lambda x: item_helper.translate(x))

In [ ]:
txns.head()

In [ ]:
cnt.head()

In [ ]:
cnt['item_id_adj'] = cnt['item_id'].map(lambda x: item_helper.translate(x))

In [ ]:
cnt.head()

### EDA

#### Checking for missing values

In [ ]:
txns.isna().sum()

In [ ]:
txns.shape

In [ ]:
cnt.isna().sum()

In [ ]:
cnt.shape

#### Checking for duplicated ratings

In [ ]:
txns.head()

In [ ]:
txns_2 = txns[['consumer_id_adj', 'item_id_adj', 'rating']]

In [ ]:
txns_2.head()

In [ ]:
duplicates = txns[txns.duplicated(subset=['consumer_id_adj', 'item_id_adj'], keep=False)]

In [ ]:
duplicates.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)

In [ ]:
duplicates.head()

##### Check whether to have sum or mean here

In [ ]:
# grp = duplicates.groupby(by=['consumer_id_adj', 'item_id_adj'])['rating'].sum()

In [ ]:
grp = duplicates.groupby(by=['consumer_id_adj', 'item_id_adj'])['rating'].mean()

In [ ]:
grp.head()

In [ ]:
grp_df = pd.DataFrame(grp)

In [ ]:
grp_df.head()

In [ ]:
grp_df.columns = ['rating_sum']

In [ ]:
grp_df.head()

In [ ]:
grp_df.reset_index(inplace=True)

In [ ]:
# grp_df.drop(columns=['rating'], inplace=True)

In [ ]:
grp_df.head()

Check distributions of ratings

In [ ]:
grp_df.describe(percentiles=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0])

In [ ]:
grp_df[grp_df['rating_sum'] == max(grp_df['rating_sum'])]

Check this consumer ID and item ID combination in original transaction table

In [ ]:
large =txns[(txns['consumer_id_adj'] == 114) & (txns['item_id_adj'] == 106)]

In [ ]:
large

In [ ]:
large['rating'].sum()

In [ ]:
txns[(txns['consumer_id_adj'] == 1) & (txns['item_id_adj'] == 53)]

In [ ]:
txns[(txns['consumer_id_adj'] == 1) & (txns['item_id_adj'] == 2)]

In [ ]:
fig = px.box(grp_df, y='rating_sum')
fig.show()

In [ ]:
no_dups = txns.drop_duplicates(subset=['consumer_id_adj', 'item_id_adj'])

In [ ]:
no_dups.head()

In [ ]:
no_dups.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)

In [ ]:
no_dups.head()

In [ ]:
duplicates.shape

In [ ]:
no_dups.shape

In [ ]:
txns.shape

In [ ]:
txns_merged = pd.merge(left=no_dups, right=grp_df, left_on=['consumer_id_adj', 'item_id_adj'], right_on=['consumer_id_adj', 'item_id_adj'], how='left')

In [ ]:
txns_merged.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)

In [ ]:
txns_merged.head(25)

In [ ]:
txns_merged['ratings_merged'] = txns_merged.rating_sum.fillna(txns_merged.rating)

In [ ]:
txns_merged.head(25)

In [ ]:
txns_merged.ratings_merged.value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
scaler = MinMaxScaler(feature_range=(1, 5))
x = scaler.fit_transform(txns_merged[['ratings_merged']]).reshape(-1)

In [ ]:
scaler_2 = StandardScaler()
y = scaler.fit_transform(txns_merged[['ratings_merged']]).reshape(-1)

In [ ]:
txns_merged['ratings_scaled'] = x

In [ ]:
txns_merged.head()

In [ ]:
# txns_merged.ratings_scaled.value_counts()

In [ ]:
txns_merged.shape

In [ ]:
txns_merged.rename(columns={'rating': 'rating_original'}, inplace=True)

In [ ]:
txns_merged.head()

In [ ]:
txns_merged.rename(columns={'ratings_scaled': 'rating'}, inplace=True)

In [ ]:
txns_merged.head()

In [ ]:
txns = txns_merged

In [ ]:
txns.describe()

#### Plotting

In [ ]:
px.histogram(txns, x='rating')

In [ ]:
px.histogram(txns, x='country')

In [ ]:
px.histogram(txns, x='country', color='rating')

In [ ]:
px.histogram(txns, x='consumer_location')

In [ ]:
cnt.head()

In [ ]:
px.histogram(cnt, x='producer_country')

In [ ]:
px.histogram(cnt, x='producer_location')

In [ ]:
px.histogram(cnt, x='item_type')

In [ ]:
px.histogram(cnt, x='language')

* When app is opened
    * Get top 10 recommendations for this user
        * If no data is available, pick top 10 from the chosen interests
        * Else, Get from the data
* When an article is read, AND one of the article recommendations is read, add it to transactions table
    * Infer the topics/genres of the article
    * Filtering should be done based on the topics/genres

Requirements
* Method to get topics/genres from content
* Generate user-article matrix
* Generate article-user matrix

### Topic Modelling using LDA

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

nltk.download('wordnet')
nltk.download('omw-1.4')

# import numpy as np
# np.random.seed(2018)

In [ ]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def lemmatize_and_stem(text):
    return stemmer.stem(lemmatizer.lemmatize(text, pos='v'))

In [ ]:
def preprocess(text):
    result = []
    for x in simple_preprocess(text):
        if x not in STOPWORDS and len(x) > 3:
            result.append(lemmatize_and_stem(x))
    return result

In [ ]:
sample_desc = cnt.iloc[0].text_description

In [ ]:
preprocess(sample_desc)

In [ ]:
processed_docs = cnt['text_description'].map(preprocess)

In [ ]:
processed_docs[:10]

In [ ]:
dic = Dictionary(processed_docs)

In [ ]:
dic.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dic.doc2bow(x) for x in processed_docs]

In [ ]:
bow_corpus[0]

In [ ]:
bow_doc_0 = bow_corpus[0]

In [ ]:
print(f'Index: {bow_doc_0[0][0]}, Word: {dic[bow_doc_0[0][0]]}, Count: {bow_doc_0[0][1]}')

In [ ]:
from gensim import corpora, models

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
bow_corpus_tfidf = tfidf[bow_corpus]

In [ ]:
from pprint import pprint
for x in bow_corpus_tfidf:
    pprint(x)
    break

In [ ]:
lda = models.LdaMulticore(bow_corpus, num_topics=10, id2word=dic, passes=2, workers=2)

In [ ]:
for idx, topic in lda.print_topics(-1):
    print(f'Topic: {idx}\n\tWords: {topic}\n')

In [ ]:
lda_tfidf = models.LdaMulticore(bow_corpus_tfidf, num_topics=10, id2word=dic, passes=2, workers=2)
for idx, topic in lda_tfidf.print_topics(-1):
    print(f'Topic: {idx}\n\tWords: {topic}\n')

### Content-based filtering

Derive keywords from the article text

In [ ]:
cnt.columns

In [ ]:
cnt.head()

In [ ]:
cnt.text_description

In [ ]:
text_array = cnt.text_description.to_numpy()

In [ ]:
# text_array

In [ ]:
cnt.shape

In [ ]:
len(text_array)

In [ ]:
words_list = []

for line in text_array:
    words = str(line).lower().split()
    words_list.append(words)

In [ ]:
len(words_list)

In [ ]:
words_list

#### Create Dictionary, Bag of Words, tfidf model & Similarity matrix

In [ ]:
%pip install gensim

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.similarities import MatrixSimilarity

# create a dictionary from words list
dictionary = Dictionary(words_list) 

#create corpus where the corpus is a bag of words for each document
corpus = [dictionary.doc2bow(doc) for doc in words_list] 

#create tfidf model of the corpus
tfidf = TfidfModel(corpus) 

# Create the similarity matrix. This is the most important part where we get the similarities between the movies.
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))


In [ ]:
def article_recommendation(content):
    #split the keywords
    keywords = content.split() 

    #set the list of words to query_doc
    query_doc = keywords 
    
    # get a bag of words from the query_doc
    query_doc_bow = dictionary.doc2bow(query_doc) 

    #convert the regular bag of words model to a tf-idf model
    query_doc_tfidf = tfidf[query_doc_bow] 

    # get similarity values between input movie and all other movies
    similarity_array = sims[query_doc_tfidf] 

    #Convert to a Series
    similarity_series = pd.Series(similarity_array.tolist()) 

    #get the most similar movies 
    # similarity_output = similarity_series.sort_values(ascending=False)
    similarity_output = similarity_series
    return similarity_output

In [ ]:
test = cnt.iloc[12]['text_description']

In [ ]:
test

In [ ]:
recs = article_recommendation(test)

In [ ]:
recs[:10]

In [ ]:
recs_df = pd.DataFrame(recs, columns=['Score'])

In [ ]:
recs_df

In [ ]:
recs_df.isna().sum()

In [ ]:
# recs_df['title'] = cnt['title']
# recs_df['text_description'] = cnt['text_description']

In [ ]:
# recs_df = cnt.join(recs_df)

In [ ]:
recs_df = cnt.join(recs_df)

In [ ]:
recs_df.head()

In [ ]:
recs_df.sort_values(by='Score', ascending=False)

In [ ]:
recs_df.isna().sum()

In [ ]:
keep = ['Score', 'title', 'text_description']

In [ ]:
recs_df.drop(columns=[col for col in recs_df if col not in keep], inplace=True)

In [ ]:
recs_df.head()

In [ ]:
recs_df.head(11)

In [ ]:
cnt.iloc[2186]

In [ ]:
txns.head()

In [ ]:
n_users = txns.consumer_id.nunique()

In [ ]:
n_articles = txns.item_id.nunique()

In [ ]:
txns.consumer_id.values

In [ ]:
print(f'Num users: {n_users}, Num articles: {n_articles}')

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(txns, test_size=0.3, random_state=42)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.describe(percentiles=[0.25, 0.5, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0])

### User-Article matrix

Since this is collaborative filtering, we will consider the transactions matrix. From this, we construct a matrix of the ratings given by users for each product.

Populate the training matrix

In [ ]:
def create_and_populate_user_article_matrix(data):
    data_matrix = np.zeros((n_users, n_articles))

    for line in data.itertuples():
        # print(line)
        # print(type(line))
        # print(f'UserId: {line.consumer_id_adj}, ArticleId: {line.item_id_adj}, Rating: {line.rating}')
        # break
        user_id = line.consumer_id_adj
        article_id = line.item_id_adj
        rating = line.rating

        data_matrix[user_id - 1, article_id - 1] = rating
    
    return data_matrix

Fill the training matrix with rating values

In [ ]:
data_matrix = create_and_populate_user_article_matrix(train)

In [ ]:
data_matrix

In [ ]:
data_matrix.shape

Populate the testing matrix

In [ ]:
data_matrix_test = create_and_populate_user_article_matrix(test)

In [ ]:
data_matrix_test

In [ ]:
data_matrix_test.shape

Create matrix with full data

In [ ]:
data_matrix_full = create_and_populate_user_article_matrix(txns)

In [ ]:
data_matrix_full

In [ ]:
data_matrix_full.shape

### Pairwise Distance

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
user_similarity = 1 - pairwise_distances(data_matrix, metric='cosine')

In [ ]:
user_similarity

In [ ]:
user_similarity.shape

Take the transpose of the data matrix in order to calculate the article similarity.

In [ ]:
# data_matrix.shape

In [ ]:
# data_matrix.T.shape

In [ ]:
article_similarity = 1 - pairwise_distances(data_matrix.T, metric='cosine')

In [ ]:
article_similarity

In [ ]:
article_similarity.shape

### Get dot product of data matrix with similarity matrix

In [ ]:
user_similarity.shape

In [ ]:
data_matrix_test.shape

In [ ]:
article_prediction = np.dot(user_similarity, data_matrix_test)

In [ ]:
article_prediction.shape

In [ ]:
article_pred_df = pd.DataFrame(article_prediction)

In [ ]:
article_pred_df.head()

In [ ]:
txns.consumer_id_adj.value_counts()

#### Test for one user

In [ ]:
test.head()

In [ ]:
test_user_id = 5
test_user_idx = test_user_id - 1

In [ ]:
test_user_id in test.consumer_id_adj

In [ ]:
article_pred_df.iloc[test_user_idx]

In [ ]:
article_recommendation = pd.DataFrame(article_pred_df.iloc[test_user_idx].sort_values(ascending=False))

In [ ]:
article_recommendation

In [ ]:
article_recommendation.reset_index(inplace=True)

In [ ]:
article_recommendation.head()

In [ ]:
article_recommendation['index'] = article_recommendation['index'] + 1

In [ ]:
article_recommendation.head()

In [ ]:
article_recommendation.rename(columns={'index': 'article_id', test_user_idx: 'score'}, inplace=True)

In [ ]:
article_recommendation.head()

Merging with the content dataframe to get the article title.

In [ ]:
merged = pd.merge(article_recommendation, cnt, left_on='article_id', right_on='item_id_adj', how='left')

In [ ]:
merged.columns

In [ ]:
keep = ['article_id', 'score', 'title']

In [ ]:
merged = merged.drop(columns=[col for col in merged if col not in keep])

In [ ]:
merged.head(10)

In [ ]:
cnt[cnt['item_id_adj'] == 2011]

In [ ]:
cnt[cnt['item_id_adj'] == 2012]

Filter out articles that have been pulled out of the system.

In [ ]:
cnt.head()

In [ ]:
cnt.interaction_type.value_counts()

In [ ]:
merged_2 = pd.merge(merged, cnt, left_on='article_id', right_on='item_id_adj', how='left')

In [ ]:
merged_2.head()

In [ ]:
merged_2.shape

In [ ]:
merged_only_present = merged_2.drop(merged_2[merged_2['interaction_type'] != 'content_present'].index)

In [ ]:
merged_only_present.shape

In [ ]:
merged.columns

In [ ]:
keep = ['article_id', 'score', 'title_x']

In [ ]:
merged_only_present.drop(columns=[col for col in merged_only_present if col not in keep], inplace=True)

In [ ]:
merged_only_present.rename(columns={'title_x': 'title'}, inplace=True)

In [ ]:
merged_only_present.head()

### Evaluate the predictions of the Collaborative User-based model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from math import sqrt

In [ ]:
data_matrix

In [ ]:
data_matrix_test

In [ ]:
article_prediction

In [ ]:
data_matrix_test_nz = data_matrix_test.nonzero()

In [ ]:
prediction = article_prediction[data_matrix_test_nz]

In [ ]:
ground_truth = data_matrix_test[data_matrix_test_nz]

#### Mean Absolute Error

In [ ]:
mean_absolute_error(prediction, ground_truth)

#### Root Mean Square Error

In [ ]:
sqrt(mean_squared_error(prediction, ground_truth))

#### Precision

In [ ]:
num_pred = 10

In [ ]:
predictions = merged_only_present['article_id'].values[:num_pred]

In [ ]:
predictions

In [ ]:
txns.head()

In [ ]:
txns[txns['consumer_id_adj'] == test_user_id]

In [ ]:
avg = txns.rating.mean()

In [ ]:
avg

In [ ]:
txns.rating.quantile(0.5)

In [ ]:
def get_articles_that_user_liked(user_id):
    user_interactions = txns[(txns['consumer_id_adj'] == user_id) & (txns['rating'] > txns.rating.mean())].sort_values(by='rating', ascending=False)

    return user_interactions['item_id_adj'].values

In [ ]:
user_interactions = txns[(txns['consumer_id_adj'] == test_user_id) & (txns['rating'] > avg)].sort_values(by='rating', ascending=False)

In [ ]:
user_interactions.shape

In [ ]:
keep = ['consumer_id_adj', 'item_id_adj', 'rating', 'ratings_merged']

In [ ]:
user_interactions.drop(columns=[col for col in user_interactions.columns if col not in keep], inplace=True)

In [ ]:
user_interactions.head(num_pred)

In [ ]:
user_interact_ids = user_interactions['item_id_adj'].values

In [ ]:
user_interact_ids

In [ ]:
get_articles_that_user_liked(5)

In [ ]:
user_interactions['item_id_adj'].nunique()

In [ ]:
set(predictions)

In [ ]:
set(user_interact_ids)

Get intersection of predictions and user interactions

In [ ]:
set(predictions) & set(user_interact_ids)

In [ ]:
precision = len(set(predictions) & set(user_interact_ids)) / num_pred

In [ ]:
precision

Recall

In [ ]:
len(set(user_interact_ids))

In [ ]:
recall = len(set(predictions) & set(user_interact_ids)) / len(set(user_interact_ids))

In [ ]:
recall

#### For sum, Precision = 0.1, Recall: 0.023255813953488372

#### For mean, Precision = 0.1, Recall: 0.043478260869565216

Some articles are correctly identified.

However, there is a massive skew in the data.

### How do we deal with skew here?

## ALS

#### Create sparse User-Article matrix

In [ ]:
from scipy.sparse import csr_matrix

Random values in CSR matrix will be filled with alpha value

In [ ]:
txns.head()

In [ ]:
keep = ['consumer_id_adj', 'item_id_adj', 'rating']

In [ ]:
txns_mod = txns.drop(columns=[col for col in txns.columns if col not in keep])

In [ ]:
txns_mod.head()

In [ ]:
txns_mod.describe()

In [ ]:
dup = txns_mod[txns.duplicated(subset=['consumer_id_adj', 'item_id_adj'])]

In [ ]:
dup.head()

In [ ]:
dup.shape

In [ ]:
txns_mod.shape[0] - dup.shape[0]

In [ ]:
txns_mod[(txns_mod['consumer_id_adj'] == 3) & (txns_mod['item_id_adj'] == 3)]

In [ ]:
alpha = 40

In [ ]:
txns_mod.shape

In [ ]:
txns_mod.shape[0]

In [ ]:
x = [alpha] * txns_mod.shape[0]

In [ ]:
len(x)

In [ ]:
sparse_user_article = csr_matrix( ([alpha]*txns_mod.shape[0], (txns_mod['consumer_id_adj'], txns_mod['item_id_adj']) ))

In [ ]:
sparse_user_article

In [ ]:
n_users

In [ ]:
n_articles

Convert to array

In [ ]:
csr_user_array = sparse_user_article.toarray()

In [ ]:
csr_user_array

In [ ]:
txns[(txns['consumer_id_adj'] == 1) & (txns['item_id_adj'] == 1)]

In [ ]:
txns.consumer_id_adj.nunique()

In [ ]:
len(csr_user_array), len(csr_user_array[0])

Dimensions match with the matrix

In [ ]:
max(csr_user_array[1])

Create article-user sparse matrix

In [ ]:
sparse_article_user = sparse_user_article.T.tocsr()

In [ ]:
sparse_article_user

Shape matches

In [ ]:
csr_article_array = sparse_article_user.toarray()

#### Create train & test data

In [ ]:
%pip install implicit

In [ ]:
from implicit.evaluation import train_test_split

In [ ]:
sparse_article_user

In [ ]:
train, test = train_test_split(sparse_user_article, train_percentage=0.8)

In [ ]:
train

In [ ]:
test

#### Building the ALS Model

In [ ]:
from implicit.als import AlternatingLeastSquares

In [ ]:
model = AlternatingLeastSquares(factors=60, regularization=0.1, iterations=60, calculate_training_loss=False)

In [ ]:
# model

Training

In [ ]:
model.fit(train)

In [ ]:
# test

In [ ]:
test_user_id = 114

In [ ]:
user_interact_ids = get_articles_that_user_liked(test_user_id)

New Implicit API expects (user, item) sparse matrix as input

In [ ]:
model.recommend(test_user_id, sparse_user_article[test_user_id], N=20, filter_already_liked_items=False)

In [ ]:
ids, scores = model.recommend(test_user_id, sparse_user_article[test_user_id], N=20, filter_already_liked_items=False)

In [ ]:
out = pd.DataFrame({'article_id': ids, 'als_score': scores})

In [ ]:
# out

In [ ]:
out.head(10)

In [ ]:
# sparse_user_article

In [ ]:
# cons = list(txns.consumer_id_adj.values)

In [ ]:
# items = list(txns.item_id_adj.values)

In [ ]:
# 1647 in cons

In [ ]:
# 1647 in items

In [ ]:
# len(user_interact_ids)

In [ ]:
# set(out.article_id)

In [ ]:
set(out.article_id)

In [ ]:
set(user_interact_ids)

In [ ]:
set(user_interact_ids) & set(out.article_id)

In [ ]:
common = set(user_interact_ids) & set(out.article_id)

In [ ]:
out[out['article_id'].isin(common)]

### Evaluation is much better with ALS.

### Getting similar articles to a given article

#### Either Content-based filtering, or Collaborative Item-based filtering, or Hybrid model

In [ ]:
article_similarity.shape

In [ ]:
n_articles

In [ ]:
article_similarity

In [ ]:
article_simi_df = pd.DataFrame(article_similarity)

In [ ]:
article_simi_df.head()

In [ ]:
test_article_id = 5

In [ ]:
test_article_index = test_article_id - 1

In [ ]:
article_simi_df.iloc[test_article_index]

In [ ]:
article_simi_df.iloc[test_article_index].sort_values(ascending=False)

In [ ]:
article_recommendation = pd.DataFrame(article_simi_df.iloc[test_article_index].sort_values(ascending=False))

In [ ]:
article_recommendation.head()

In [ ]:
article_recommendation.reset_index(inplace=True)

In [ ]:
article_recommendation.head()

In [ ]:
article_recommendation.rename(columns={'index': 'article_id', test_article_index: 'similarity_score'}, inplace=True)

In [ ]:
article_recommendation.head()

Convert the IDs to 1-indexed

In [ ]:
# Convert article_recommendation.article_id from zero-based index to one-based index
article_recommendation['article_id'] = article_recommendation['article_id'] + 1

In [ ]:
article_recommendation.head()

In [ ]:
get_articles_that_user_liked(test_user_id)

In [ ]:
test_user_id

Filter out articles that the user has already read.

### Article-User Matrix

Same as User-Article matrix